In [378]:
import itertools

import numpy as np
import pandas as pd

In [553]:
def clean_integrals(Z, file_name):

    df = pd.read_csv(file_name, header=None)
    integrals_dict = {}

    for row in df.iterrows():

        key, val = row[1][0].split('=')

        key = key.replace('<', '')
        key = key.replace('>', '')
        key = key.replace('|V|', '')
        key = key.replace(' ', '')

        val = val.replace('[', '(')
        val = val.replace(']', ')')
        val = val.replace('Sqrt', 'np.sqrt')
        integrals_dict.update({f'{key}': eval(val)})

    return integrals_dict

In [554]:
integrals = clean_integrals(2, 'integrals.csv')


{'1111': 1.25,
 '1112': 0.17871006683882326,
 '1113': 0.08791889899219621,
 '1121': 0.17871006683882326,
 '1122': 0.0438957475994513,
 '1123': 0.02244583699658215,
 '1131': 0.08791889899219621,
 '1132': 0.02244583699658215,
 '1133': 0.01153564453125,
 '1211': 0.17871006683882326,
 '1212': 0.41975308641975306,
 '1213': 0.10105295382557257,
 '1221': 0.0438957475994513,
 '1222': 0.017163314819200585,
 '1223': 0.009085693750900134,
 '1231': 0.02244583699658215,
 '1232': 0.00787190185136582,
 '1233': 0.004218077669259728,
 '1311': 0.08791889899219621,
 '1312': 0.10105295382557257,
 '1313': 0.198974609375,
 '1321': 0.02244583699658215,
 '1322': 0.009085693750900134,
 '1323': 0.00485770635685417,
 '1331': 0.01153564453125,
 '1332': 0.004218077669259728,
 '1333': 0.0022622638577071527,
 '2111': 0.17871006683882326,
 '2112': 0.0438957475994513,
 '2113': 0.02244583699658215,
 '2121': 0.41975308641975306,
 '2122': 0.017163314819200585,
 '2123': 0.00787190185136582,
 '2131': 0.10105295382557257,
 

In [545]:
def jacobi(A, eps=1e-15, maxiter=10000):
    """
    Jacobi method for diagonalising a symmetric matrix A
    """
    n = A.shape[0]
    V = np.eye(n)
    for i in range(maxiter):
        # find the largest off-diagonal element
        maxval = 0
        for j in range(n):
            for k in range(j+1, n):
                if abs(A[j,k]) > maxval:
                    maxval = abs(A[j,k])
                    p = j # row index
                    q = k # column index
        if maxval < eps:
            break
        # rotate the matrix
        theta = 0.5 * np.arctan2(2*A[p,q], A[p,p]-A[q,q]) # rotation angle
        c = np.cos(theta)
        s = np.sin(theta)
        R = np.eye(n)
        R[p,p] = c
        R[p,q] = -s
        R[q,p] = s
        R[q,q] = c
        A = R.T @ A @ R # matrix rotation
        V = V @ R 
    return A, V # diagonalised matrix and eigenvectors


In [546]:
hole_states = ['1+', '1-']
particle_states = ['2+', '2-', '3+', '3-']

def matrix_config(hole_states, particle_states):
    excitations = list(itertools.product(hole_states, particle_states))
    excitations = [list(x) for x in excitations if x[0][1]==x[1][1]]  ## removing non-zero-spin excitations

    bra = ket = [['0','0']] + excitations  ## ground state and excitations

    elements = list(itertools.product(bra, ket))
    elements = [list(element) for element in elements]

    matrix = np.zeros((5,5), dtype = object)        ## doing this because numpy reshape is acting weird
    row = 0
    for i, element in enumerate(elements):
        col = i % 5
        matrix[row, col] = element
        if col == 4 and i != 0:
            row += 1

    return matrix

In [547]:
def h0_term(alpha, beta, Z):
    if not alpha == beta:
        return 0

    n= int(alpha[0])
 
    return -(Z**2)/(2*n**2)

def v_term(alpha, beta, gamma, delta):
    alpha_n, alpha_spin = alpha[0], alpha[-1]
    beta_n, beta_spin = beta[0], beta[-1]
    gamma_n, gamma_spin = gamma[0], gamma[-1]
    delta_n, delta_spin = delta[0], delta[-1]

    direct_term = integrals[alpha_n+beta_n+gamma_n+delta_n]
    exchange_term = integrals[alpha_n+beta_n+delta_n+gamma_n]

    if alpha_spin != gamma_spin or beta_spin != delta_spin:
        direct_term = 0

    if alpha_spin != delta_spin or beta_spin != gamma_spin:
        exchange_term = 0

    return direct_term - exchange_term

def below_fermi(fermi_level):
    n = [str(n) for n in range(1, fermi_level + 1)] 
    spin = ["+", "-"] 
    return  list(itertools.product(n, spin))


def zp_zh(Z, fermi_level):
    cumul_h0 = 0
    cumul_v = 0

    states_below_fermi = below_fermi(fermi_level)

    #cumul_h0
    for i in states_below_fermi:
        cumul_h0 += h0_term(i, i, Z)

    #cumul_v
    for i in states_below_fermi:
        for j in states_below_fermi:
            cumul_v += v_term(i, j, i, j)

    return cumul_h0 + (1/2)*cumul_v

def onep_oneh(Z, fermi_level, a, i, b, j):
    cumul_h0 = 0
    cumul_v = 0
    states_below_fermi = below_fermi(fermi_level)

    if a==b and i==j and a[0]!= '0' :   ## diagonal
        cumul_h0 += h0_term(a, a, Z)
        for j in states_below_fermi:
            cumul_v += v_term(a, j, a, j)

        cumul_h0 += - h0_term(i, i, Z)
        for j in states_below_fermi:
            cumul_v += - v_term(i, j, i, j)

        return cumul_h0 + cumul_v + zp_zh(Z, fermi_level) ## this last term is E_0 ref
    
    # from core to excitation
    elif (a[0]==i[0]=='0') or (b[0]==j[0]=='0'): ## notice that the case where all are equal to zero will not happend because it is not passed to 1p1h

        if (b[0]=='0'): # than we change. we can do this because the matrix is symetric
            b, j = a, i
        for i in states_below_fermi:
            cumul_v +=  v_term(j, i, b, i) 

        return cumul_v

    else:
        return v_term(a, j, i, b) 

In [548]:
matrix = matrix_config(hole_states, particle_states)

for row in range(0,5):
    for col in range(0, 5):
        i = matrix[row, col][0][0]
        a = matrix[row, col][0][1] 
        j = matrix[row, col][1][0]
        b = matrix[row, col][1][1]

        if a == i == j == b== '0':    ## 0p0h 0p0h
            matrix[row, col] = zp_zh(2, 1)

        else:
            matrix[row, col] = onep_oneh(2, 1, a, i, b, j)

In [552]:
strin = ''

for i in range(0,5):
    for j in range(0, 5):
        strin += f"{matrix[i, j]: 6.3f} "

    strin += '\n'

print(strin)


-2.750  0.179  0.088  0.179  0.088 
 0.179 -1.704 -0.079  0.044  0.022 
 0.088 -0.079 -1.836  0.022  0.012 
 0.179  0.044  0.022 -1.704 -0.079 
 0.088  0.022  0.012 -0.079 -1.836 



In [542]:
jacobi(matrix)


(array([[-1.6009489749568355, -6.464122800409762e-27,
         -4.765465477458209e-31, 1.0055439021128555e-20,
         -1.6867738846926308e-17],
        [3.252612686413782e-17, -1.6852765056564138,
         1.314889332557839e-20, -1.730608552684508e-24,
         2.524354896707238e-29],
        [-2.6886465226870966e-17, 2.4966848596223537e-16,
         -1.8101957827089663, 0.0, -1.3202460420942341e-23],
        [-3.192562745034552e-17, -2.817113055320757e-17,
         1.665335526087584e-16, -1.9103531092377046,
         2.9632331697696546e-30],
        [-6.1143724132947e-17, -8.361802517087588e-18,
         3.5582186665193695e-17, 9.101319540891532e-17,
         -2.823622072966912]], dtype=object),
 array([[ 1.99119479e-01, -3.59606680e-16,  1.57517325e-01,
         -1.95542822e-16, -9.67233025e-01],
        [ 6.86507287e-01,  6.00023385e-01,  7.25063934e-02,
          3.74128237e-01,  1.53135783e-01],
        [-9.42521171e-02, -3.74128237e-01,  6.94504837e-01,
          6.00023385e-01